### 3. Generación de Lenguaje Natural
Implementación de un modelo de generación de lenguaje utilizando algún dataset de los propuestos de Twitter.

El objetivo es el de simular la creación de un bot que escriba tweets de manera similar al del autor utilizado para el entrenamiento.

Se valorará principalmente la argumentación de los pasos realizados y las conclusiones al analizar los resultados y limitaciones encontradas.

In [10]:
# Importamos a lo que vayamos a necesitar

import numpy as np
import pandas as pd
import re
from copy import deepcopy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [3]:
from tensorflow.python.client import device_lib

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6917121431308361240
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1612668119331153853
physical_device_desc: "device: XLA_CPU device"
]


### Cargamos los datos

In [46]:
data = pd.read_csv('./datasets/KimKardashianTweets.csv')

In [47]:
data.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 2,782694393216110592,/KimKardashian/status/782694393216110592,False,Hey guys! Meet my #KimKardashianGame character...,KimKardashian
1,1,Oct 2,782632831780618240,/KimKardashian/status/782632831780618240,False,This guy is always in my shot!pic.twitter.com/...,KimKardashian
2,2,Oct 2,782559825205071872,/KimKardashian/status/782559825205071872,False,Balenciaga. No make up today.pic.twitter.com/l...,KimKardashian
3,3,Oct 1,782260907900567552,/KimKardashian/status/782260907900567552,False,Kourtney and Kim Take Parispic.twitter.com/VXw...,KimKardashian
4,4,Oct 1,782259209459101696,/KimKardashian/status/782259209459101697,False,GIVENCHY COUTUREpic.twitter.com/jHxTIuHzzo,KimKardashian


In [48]:
data = data[['retweet', 'text', 'author']]

In [49]:
data.head()

,retweet,text,author
0,False,Hey guys! Meet my #KimKardashianGame character...,KimKardashian
1,False,This guy is always in my shot!pic.twitter.com/...,KimKardashian
2,False,Balenciaga. No make up today.pic.twitter.com/l...,KimKardashian
3,False,Kourtney and Kim Take Parispic.twitter.com/VXw...,KimKardashian
4,False,GIVENCHY COUTUREpic.twitter.com/jHxTIuHzzo,KimKardashian


In [114]:
aux = 'Hey guys! Meet my #KimKardashianGame character at the So Chic Boutique in Downtown LA for a surprise ;)'

In [115]:
aux.replace('Meet', 'caca')

'Hey guys! caca my #KimKardashianGame character at the So Chic Boutique in Downtown LA for a surprise ;)'

In [117]:
hashtag_regex.subn(hashtag_regex.pattern, ' ', aux)

TypeError: 'str' object cannot be interpreted as an integer

Pasos a seguir 

- extraer los hashtags, menciones y  urls
- convertirlos a un array

In [73]:
regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar'
regex.sub(lambda m: m.group().replace('"',"%",1), myfile)

'foo%s bar'

In [107]:
user_regex = re.compile('@\w+ | @\w+ ')
hashtag_regex = re.compile('#\w+| #\w+ ')
url_regex = re.compile('[http|https]?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')
pics_regex = re.compile('pic?.\w+.\w+\/\w+')

In [ ]:
review = re.sub(r'(&#\d+) | (&\w+)', '', review)

In [84]:
pics_regex.pattern

'pic?.\\w+.\\w+\\/\\w+'

In [130]:
users = []
hashtags = []
urls = []
pics = []


texts = []
for text in data['text'][:100]:
    
    this_usrs = user_regex.findall(text)
    this_hashtag = hashtag_regex.findall(text)
    this_url = url_regex.findall(text)
    this_pics = pics_regex.findall(text)
    
    aux = text
    aux = re.sub(user_regex.pattern, ' ', aux)
    aux = re.sub(hashtag_regex.pattern, ' ', aux)
    aux = re.sub(url_regex, '', aux)
    aux = re.sub(pics_regex, '', aux)
    
    texts.append(aux)
    

In [132]:
texts

['Hey guys! Meet my character at the So Chic Boutique in Downtown LA for a surprise ;)',
 'This guy is always in my shot!',
 'Balenciaga. No make up today.',
 'Kourtney and Kim Take Paris',
 'GIVENCHY COUTURE',
 'The Yeezy Show Room in Paris is heaven.',
 "Think I'm gonna wear no make up today to the Balenciaga show.",
 "Just finished the Amanda Knox doc. I don't think they did it. I really want a Natalie Halloway doc & they need to solve that crime. So sad!",
 'Paris Squad',
 'Came home early from the fashion festivities to watch the Netflix Amanda Knox doc. Cannot wait to see this! Anyone see it yet? Thoughts?',
 'Hair flip & eye roll at the same damn time!',
 'Sister Slay',
 'On the way to Balmain',
 'My security   is a G',
 'NYC  PARIS',
 'Currently  .twitter.com/QVhNFEa4TH',
 'He is desperately searching for a bone marrow match to survive, and you could be that match!!',
 'He is 31 and a father of three who was recently diagnosed with an aggressive blood cancer called primary myel

In [143]:
text = r'\n\n'.join(texts)

In [144]:
text

'Hey guys! Meet my character at the So Chic Boutique in Downtown LA for a surprise ;)\\n\\nThis guy is always in my shot!\\n\\nBalenciaga. No make up today.\\n\\nKourtney and Kim Take Paris\\n\\nGIVENCHY COUTURE\\n\\nThe Yeezy Show Room in Paris is heaven.\\n\\nThink I\'m gonna wear no make up today to the Balenciaga show.\\n\\nJust finished the Amanda Knox doc. I don\'t think they did it. I really want a Natalie Halloway doc & they need to solve that crime. So sad!\\n\\nParis Squad\\n\\nCame home early from the fashion festivities to watch the Netflix Amanda Knox doc. Cannot wait to see this! Anyone see it yet? Thoughts?\\n\\nHair flip & eye roll at the same damn time!\\n\\nSister Slay\\n\\nOn the way to Balmain\\n\\nMy security   is a G\\n\\nNYC  PARIS\\n\\nCurrently  .twitter.com/QVhNFEa4TH\\n\\nHe is desperately searching for a bone marrow match to survive, and you could be that match!!\\n\\nHe is 31 and a father of three who was recently diagnosed with an aggressive blood cancer c

In [145]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

print('Unique chars: {}'.format(len(characters)))

Unique chars: 81


In [146]:
print(characters)

[' ', '!', '"', '&', "'", ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '\\', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', 'é', '‘', '’', '…']


In [147]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [148]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [149]:
# Model 4
model_4 = Sequential()
model_4.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model_4.add(Dropout(0.2))
model_4.add(LSTM(700))
model_4.add(Dropout(0.2))
model_4.add(Dense(Y_modified.shape[1], activation='softmax'))
model_4.compile(loss='categorical_crossentropy', optimizer='adam')

model_4.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 700)          1965600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 700)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 700)               3922800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 81)                56781     
Total params: 5,945,181
Trainable params: 5,945,181
Non-trainable params: 0
_________________________________________________________________


In [153]:
history = model_4.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Instructions for updating:
Use tf.cast instead.
Train on 6550 samples, validate on 345 samples
Epoch 1/20
 512/6550 [=>............................] - ETA: 3:19 - loss: 4.2963

KeyboardInterrupt: 